# Ch 02: Concept 09

## using Queues

만약 트레이닝 데이터가 많다고 한다면 이것을 모두 한번에 불러오는 것은 힘든 일이다. 

텐서플로우에서 QueueRunner는 효율적인 도구이다. Queue data structure를 통해서 이를 수행할 수 있다. 

In [1]:
import tensorflow as tf
import numpy as np

multiple threads를 사용하겠다. 사용할 CPU의 수를 명시해야 한다.

In [2]:
import multiprocessing
NUM_THREADS = multiprocessing.cpu_count() # 4

예제에 사용할 데이터를 생성한다.

In [3]:
xs = np.random.randn(100, 3)
ys = np.random.randint(0, 2, size=100)

여기 우리 데이터에 대한 예제가 있다.

In [4]:
xs_and_ys = zip(xs, ys)
for _ in range(5):
    x, y = next(xs_and_ys)
    print('Input {}  --->  Output {}'.format(x, y))

Input [ 0.03459827  0.85180779  1.16363823]  --->  Output 1
Input [ 1.86116909 -2.63707184 -0.24510653]  --->  Output 0
Input [ 0.32256414 -0.54448053  2.23311806]  --->  Output 0
Input [-0.59111381  0.97370897  1.82711469]  --->  Output 0
Input [ 0.4993818   0.86559326 -0.52467918]  --->  Output 0


queue를 define한다.

In [5]:
queue = tf.FIFOQueue(capacity=1000, dtypes=[tf.float32, tf.int32])

enqueue와 dequeue를 set up한다.

In [7]:
enqueue_op = queue.enqueue_many([xs, ys])
x_op, y_op = queue.dequeue()

queue runner 를 define한다. 

In [8]:
qr = tf.train.QueueRunner(queue, [enqueue_op] * 4)

이제 모든 변수와 ops에 대한 설정이 완료되었다. 세션을 시작해보자.

In [9]:
sess = tf.InteractiveSession()

queue runner에 대한 쓰레드를 생성하자.

In [10]:
coord = tf.train.Coordinator()
enqueue_threads = qr.create_threads(sess, coord=coord, start=True)

우리의 dequeue를 test 하자.

In [11]:
for _ in range(100):
    if coord.should_stop():
        break
    x, y = sess.run([x_op, y_op])
    print(x, y)
coord.request_stop()
coord.join(enqueue_threads)

[ 0.03459826  0.85180777  1.16363823] 1
[ 1.8611691  -2.63707185 -0.24510653] 0
[ 0.32256415 -0.5444805   2.23311806] 0
[-0.59111381  0.97370899  1.8271147 ] 0
[ 0.49938181  0.86559325 -0.52467918] 0
[ -3.00911081e-04   1.24408376e+00  -2.68503446e-02] 0
[-0.32003999  0.30943733  1.17295301] 0
[-0.74158692  0.53319955  0.2528984 ] 1
[ 0.38695502  1.94423437  1.06784046] 0
[ 1.63434303  0.93925041 -0.6698125 ] 1
[ 0.10698817 -1.31199956  0.01201056] 0
[ 1.73152852  1.77715683 -1.29911458] 0
[-1.0566355   1.1012038   0.00128921] 0
[-0.47042507 -0.79420763  0.46603519] 0
[-1.37130868 -0.07138771 -0.91868263] 0
[-1.04607069  0.50373828  0.5067777 ] 0
[-0.38719022  0.0960427  -0.74320936] 0
[ 0.20445429 -0.26628113 -1.42132127] 1
[ 0.0744261   1.12379897 -0.43687195] 1
[-0.38301447  1.14882672 -0.70178807] 0
[-0.88415366 -2.10833216 -0.88596445] 1
[-0.53213567  0.49527898  1.02490401] 0
[-1.22664678 -0.55283856  0.21844564] 0
[ 0.20615141 -0.43299392  0.28879598] 0
[ 0.5992589   1.51104307 